#Installation and imports

In [ ]:
%%capture
%%bash
# Install RDKit and xTB. Takes 2-3 minutes
pip install rdkit-pypi
pip install py3Dmol
wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
chmod +x Miniconda3-latest-Linux-x86_64.sh
bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
#conda install -q -y -c conda-forge python=3.7
#conda install -q -y -c conda-forge rdkit=2020.09.2
conda install -q -y -c conda-forge xtb

In [ ]:
# Much faster RDKit install if you don't need xTB
#%%capture
#!pip install rdkit-pypi
#!pip install py3Dmol

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from rdkit import Chem
from rdkit.Chem import AllChem
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDistGeom
IPythonConsole.ipython_3d = True

import py3Dmol
import matplotlib.pyplot as plt
import subprocess

#Functions

These functions are also available [here](https://github.com/jensengroup/rdkit_qm_utilities)

##write_xyz

In [ ]:
def write_xyz(mol, file_name='temp.xyz'):
  number_of_atoms = mol.GetNumAtoms()
  symbols = [a.GetSymbol() for a in mol.GetAtoms()] 
  with open(file_name, "w") as file:
    file.write(str(number_of_atoms)+"\n")
    file.write("title\n")
    conf = mol.GetConformers()[0]
    for atom,symbol in enumerate(symbols):
      p = conf.GetAtomPosition(atom)
      line = " ".join((symbol,str(p.x),str(p.y),str(p.z),"\n"))
      file.write(line)

##show_mol

In [ ]:
def show_mol(file_name, animate=False):
  xyz=open(file_name, 'r').read()
  p = py3Dmol.view(width=400,height=400)
  if animate:
    p.addModelsAsFrames(xyz,'xyz')
    p.animate({'loop': "forward",'reps': 5})
    #p.animate({'loop': 'backAndForth'})
  else:
    p.addModel(xyz,'xyz')
  p.setStyle({'stick':{}})
  p.setBackgroundColor('0xeeeeee')
  p.zoomTo()
  p.show()

##get_best_structure

In [ ]:
def get_best_structure(mol,n_confs=10):
  new_mol = Chem.Mol(mol)

  AllChem.EmbedMultipleConfs(mol,numConfs=n_confs,useExpTorsionAnglePrefs=True,useBasicKnowledge=True)
  energies = AllChem.MMFFOptimizeMoleculeConfs(mol,maxIters=2000, nonBondedThresh=100.0)

  energies_list = [e[1] for e in energies]
  min_e_index = energies_list.index(min(energies_list))

  new_mol.AddConformer(mol.GetConformer(min_e_index))

  return new_mol

##shell

In [ ]:
def shell(cmd, shell=False):
  #Written by Jimmy Kromann
  if shell:
    p = subprocess.Popen(cmd, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  else:
    cmd = cmd.split()
    p = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

  output, err = p.communicate()
  return output

##run_xTB

In [ ]:
def run_xTB(mol, command, file_name='temp.xyz', conf_search=False):
  mol = Chem.AddHs(mol)
  if conf_search:
    mol = get_best_structure(mol)
  else:
    rdDistGeom.EmbedMolecule(mol)
    AllChem.MMFFOptimizeMolecule(mol)
  write_xyz(mol)
  command = 'xtb ' + file_name + ' ' + command
  output = shell(command)
  output = str(output).replace('\\n','\n')
  
  return output

##get_energy

In [ ]:
def get_energy(output):
  output = str(output)
  energy = float(output.split('TOTAL ENERGY')[1].split('Eh')[0])

  return energy

##run_rxn

In [ ]:
 def run_rxn(reactant, smarts): 
  rxn = AllChem.ReactionFromSmarts(smarts)
  ps = rxn.RunReactants((reactant,))
  product = ps[0][0]
  Chem.SanitizeMol(product)
  
  return product

##reorder_product

In [ ]:
def reorder_product(product):
  reorder_inverse = [int(atom.GetProp('react_atom_idx')) for atom in product.GetAtoms()]
  reorder = len(reorder_inverse)*[0]

  for i in range(len(reorder_inverse)):
    reorder[reorder_inverse[i]] = i

  product = Chem.RenumberAtoms(product, reorder)

  return product

##label_atoms

In [ ]:
def label_atoms(mol):
  for i,atom in enumerate(mol.GetAtoms()):
    atom.SetAtomMapNum(i+1)
  
  return mol

##plot_PES

In [ ]:
def plot_PES():
  output = shell('grep energy xtbpath.xyz')
  energies = []
  for line in str(output).split('\\n'):
    if 'energy:' in line:
      energies.append(float(line.split('energy:')[1].split('xtb:')[0]))

  plt.plot(energies,'bo')
  plt.ylabel('Relative energy (kcal/mol)')
  plt.xlabel('Interpolation step')

##offset

In [ ]:
def offset(mol, offset=5):
  smiles = Chem.MolToSmiles(mol, allHsExplicit=True)
  smilesA = smiles.split('.')[0]
  molA_idx = mol.GetSubstructMatch(Chem.MolFromSmiles(smilesA, sanitize=False))
  conf = mol.GetConformer()
  for i in range(mol.GetNumAtoms()):
    p = conf.GetAtomPosition(i)
    if i in molA_idx:
      conf.SetAtomPosition(i,(p.x,p.y,p.z+offset))

  return mol

#Some basics: geometry optimisation and getting the energy from the output

## Geometry optimisation

[PubChem Sketcher](https://pubchem.ncbi.nlm.nih.gov/edit3/index.html): a GUI for SMILES generation

Getting SMILES from [ChemDraw](https://youtu.be/A_gwCZrYIXY)

##Getting a mol object/smiles from an xyz file


In [ ]:
%%capture
!git clone https://github.com/jensengroup/xyz2mol.git
import xyz2mol.xyz2mol as xyz2mol

In [ ]:
#atoms, charge, xyz_coordinates = xyz2mol.read_xyz_file('xtbopt.xyz')
#mols = xyz2mol.xyz2mol(atoms, xyz_coordinates, charge=charge)
#mol = Chem.RemoveHs(mols[0])
#Chem.MolToSmiles(mol)

## Getting the energy from the output

The energy unit (Eh) is Hartree. 1 Hartree = 627.51 kcal/mol = 2625.08 kJ/mol

#Reaction energies: pKa prediction

$HA^+\rightarrow A + H^+$  and $pK_a = \frac{\Delta G^\circ}{RT\ln(10)}$ but $\Delta G^\circ(H^+)=?$

Instead: 

$HA^+ + A_{\mathrm{ref}}\rightarrow A + AH^+_{\mathrm{ref}}$  and $\Delta pK_a = \frac{\Delta G^\circ}{RT\ln(10)}$

where $\Delta G^\circ \approx \Delta E$

[Some pKa values](https://www.cambridgemedchemconsulting.com/resources/tuning_bases.html).  [Paper](https://dx.doi.org/10.1021/acs.jpca.6b10990).

#Barrier estimation

xTB has a quick and dirty [reaction path estimator](https://xtb-docs.readthedocs.io/en/latest/path.html) ([paper1](https://doi.org/10.1021/acs.jctc.9b00143), [paper2](https://doi.org/10.7717/peerj-pchem.15)).

The following energy expression is minimized

$E=E+k_{push} e^{-\alpha \Delta_r^{2}}+k_{pull} e^{-\alpha \Delta_p^{2}}$

where $k_{push}>0$ and $k_{pull}<0$ and $\Delta_r$ and $\Delta_p$ are the RMSDs between the current structure and the reactant and product, respectively.

The reaction path consists of points along the energy minisation path, with the biasing potential removed.

Note that the atom order must be the same in the reactants and products.

## Reaction smarts

##atom_mapper

In [ ]:
%%capture
!git clone https://github.com/jensengroup/atom_mapper.git